<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Interactive-Visualization-Lab" data-toc-modified-id="Interactive-Visualization-Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Interactive Visualization Lab</a></span><ul class="toc-item"><li><span><a href="#1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011." data-toc-modified-id="1.-Create-an-interactive-bar-chart-showing-total-quantity-and-revenue-by-country-(excluding-United-Kingdom)-for-the-month-of-April-2011.-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.</a></span></li><li><span><a href="#2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011." data-toc-modified-id="2.-Create-an-interactive-line-chart-showing-quantity-and-revenue-sold-to-France-between-January-1st-and-May-31st-2011.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.</a></span></li><li><span><a href="#3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories)." data-toc-modified-id="3.-Create-an-interactive-scatter-plot-showing-the-relationship-between-average-quantity-(x-axis)-and-average-unit-price-(y-axis)-for-the-product-PARTY-BUNTING-with-the-plot-points-color-coded-by-country-(categories).-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).</a></span></li><li><span><a href="#4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands." data-toc-modified-id="4.-Create-a-set-of-interactive-histograms-showing-the-distributions-of-quantity-per-invoice-for-the-following-countries:-EIRE,-Germany,-France,-and-Netherlands.-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.</a></span></li><li><span><a href="#5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below." data-toc-modified-id="5.-Create-an-interactive-side-by-side-bar-chart-showing-the-revenue-by-country-listed-below-(bars)-for-each-of-the-products-listed-below.-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.</a></span></li><li><span><a href="#6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart." data-toc-modified-id="6.-Create-an-interactive-line-chart-showing-quantity-sold-by-day-for-the-United-Kingdom.-Add-drop-down-boxes-for-Year-and-Month-that-allow-you-to-filter-the-date-range-that-appears-in-the-chart.-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.</a></span></li><li><span><a href="#7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges." data-toc-modified-id="7.-Create-an-interactive-scatter-plot-that-plots-number-of-invoices-(x-axis)-vs.-number-of-customers-(y-axis)-and-the-plot-points-represent-individual-products.-Add-two-sliders-that-control-the-x-and-y-axis-ranges.-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.</a></span></li><li><span><a href="#8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description." data-toc-modified-id="8.-Creat-an-interactive-bar-chart-that-shows-revenue-by-product-description.-Add-a-text-field-widget-that-filters-the-results-to-show-the-product-that-contain-the-text-entered-in-their-description.-1.8"><span class="toc-item-num">1.8&nbsp;&nbsp;</span>8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.</a></span></li></ul></li></ul></div>

# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [1]:
import pandas as pd
import plotly.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

In [2]:
data = pd.read_excel('../data/Online Retail.xlsx')

In [3]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [4]:
a = data.loc[(data['Country']!='United Kingdom') & (data['InvoiceDate']>'2011-04-01 00:00:00') & (data['InvoiceDate']<'2011-04-30 23:59:59')]
b = a[['Quantity', 'Revenue']].groupby(data['Country']).sum()

b.iplot(kind='bar', title='Quantity and Revenue per country (except UK) in April 2011')

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [5]:
c = data.loc[(data['Country'] == 'France') & (data['InvoiceDate'] < '2011-05-31') & (data['InvoiceDate'] > '2011-01-01')]
d = c[['Quantity', 'Revenue']].groupby(c['InvoiceDate']).sum()

d.iplot(kind='line', title='Quantity and Revenue sold to France Jan-May 2011')

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [86]:
e = data.loc[data['Description']=='PARTY BUNTING']
e = e[['Quantity', 'UnitPrice', 'Country']]
e.head()
e.iplot(x='Quantity', y='UnitPrice', categories='Country', xTitle='Avg Quantity', yTitle='Avg Unit Price', title='Party Bunting Price per Unit')

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [87]:
country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

r = data[['InvoiceNo', 'Country', 'Quantity']]

r2 = r.loc[r['Country'].isin(country_list)]

r3 = r2.pivot_table(values='Quantity', columns='Country', index='InvoiceNo', aggfunc='sum')

r3.iplot(kind='hist', subplots=True, shape=(2,2), title='Quantity per Invoice', xTitle='Quantity', yTitle='Invoices')

## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [8]:
data.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom


In [63]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [67]:
rev2 = data.loc[rev['Country'].isin(country_list)]
rev3 = rev2.loc[rev2['Description'].isin(product_list)]

rev4 = rev3.pivot_table(values='Revenue', index='Country', columns='Description', aggfunc='sum')
rev4

Description,CREAM HANGING HEART T-LIGHT HOLDER,JUMBO BAG RED RETROSPOT,REGENCY CAKESTAND 3 TIER
Country,,,
EIRE,2740.80,278.72,7388.55
France,131.75,903.37,2816.85
Germany,35.40,1072.76,9061.95
Netherlands,1167.00,3468.00,3166.35


In [85]:
rev4.iplot(kind='bar', subplots=True, shape=(1,3), title='Revenue by Country of the products', xTitle='Country', yTitle='Revenue')


## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [11]:
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']

In [12]:
uk.head()

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.3,17850,United Kingdom,2010,12,1
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.2,17511,United Kingdom,2010,12,1
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.6,13408,United Kingdom,2010,12,1


In [26]:
@interact(Selection=['Month', 'Day'])

def linechart(Selection):
    uk2 = uk.groupby([Selection], as_index=False).agg({'Quantity':'sum'})

    uk2.iplot(kind='line', x=Selection, xTitle=Selection.title(), yTitle='Quantity', title='Quantity per '+Selection.title())
    

interactive(children=(Dropdown(description='Selection', options=('Month', 'Day'), value='Month'), Output()), _…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [38]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description', as_index=False).agg(agg_func)

In [39]:
products.head(10)

,Description,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
0,4 PURPLE FLOCK DINNER CANDLES,35,134,2.318421,255.46,30
1,50'S CHRISTMAS GIFT BAG LARGE,100,1721,1.247900,2067.25,98
2,DOLLY GIRL BEAKER,100,661,1.250000,826.25,77
3,I LOVE LONDON MINI BACKPACK,55,181,4.150000,751.15,46
4,NINE DRAWER OFFICE TIDY,25,44,14.761538,628.40,24
5,OVAL WALL MIRROR DIAMANTE,55,112,9.842857,970.40,39
6,RED SPOT GIFT BAG LARGE,91,1652,1.247742,1981.00,82
7,SET 2 TEA TOWELS I LOVE LONDON,337,4200,3.121064,11910.80,168
8,SPACEBOY BABY GIFT SET,93,181,16.694681,2925.95,76
9,TOADSTOOL BEDSIDE LIGHT,88,184,8.942308,1642.60,70


In [54]:
products.describe().T

,count,mean,std,min,25%,50%,75%,max
InvoiceNo,3620.0,95.035635,143.765886,1.000000,12.000000,43.00,119.000000,1891.000000
Quantity,3620.0,1131.021271,2588.849272,1.000000,59.000000,325.00,1134.250000,49182.000000
UnitPrice,3620.0,3.368378,6.982657,0.042807,0.973333,1.95,3.812449,158.913043
Revenue,3620.0,1939.284262,4884.328140,0.420000,113.377500,512.55,1727.197500,110990.200000
CustomerID,3620.0,65.665193,82.995934,1.000000,10.000000,36.00,89.000000,821.000000


In [88]:

@interact(x_range = (1, max(products['InvoiceNo'])), y_range = (1,max(products['CustomerID'])))    

def scatter(x_range, y_range):
    
    fig = products.iplot(kind='scatter', x='InvoiceNo', y='CustomerID', categories='Description',
                         title='Invoices per customer', xTitle='Number of invoices', yTitle='Number of customers')
    fig.update_xaxes(range=[0,x_range])
    fig.update_yaxes(range=[0,y_range])

#no consigo que el rango de los ejes se regule mediante los sliders

interactive(children=(IntSlider(value=946, description='x_range', max=1891, min=1), IntSlider(value=411, descr…

## 8. Creat an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.

In [60]:

@interact(Description='')
def chart(Description):
    data_text = data[data['Description'].str.contains(Description)]
    data_text = data_text.groupby('Description', as_index=False).agg({'Revenue':'sum'})
    
    data_text.iplot(kind='bar', x='Description', y='Revenue', title='Revenue per Product')

interactive(children=(Text(value='', description='Description'), Output()), _dom_classes=('widget-interact',))